### María Sofía Álvarez, Brenda Barahona, Álvaro Plata
<h1 align='center'>Proyecto 1: Analítica de textos - Transfer Learning</h1>

## Instalación de librerías
<b><font color='blue'>Importante:</font></b> Correr antes de ejecutar el notebook. Con una única vez que se corra, basta.

In [35]:
# !pip install -q keras-bert
#!pip install keras-rectified-adam
#!pip install transformers
# !pip install torch

In [50]:
!export NCBI_DIR='/Users/sofiaalvarezlopez/Documents/Universidad/Décimo Semestre/Inteligencia de Negocios/Proyectos/Proyecto 1/NCBI_BERT_pubmed_mimic_uncased_L-12_H-768_A-12'

In [72]:
import os
import keras
import torch
import codecs
import numpy as np
import pandas as pd
import tensorflow as tf
from keras_radam import RAdam
from keras_bert import Tokenizer
from transformers import BertModel
from transformers import BertTokenizer
tf.compat.v1.disable_eager_execution()
from tensorflow.keras.optimizers import Adam
from keras_bert import load_trained_model_from_checkpoint

## Transfer Learning
El transfer learning es una de las aplicaciones de la inteligencia artificial que más fuerza ha cogido últimamente. Esta se enfoca en almacenar conocimiento obtenido al resolver un problema determinado, y reutilizarlo para aplicarlo a un problema diferente, pero relacionado.

La evolución de los modelos preentrenados en los últimos años ha facilitado enormemente el desarrollo de modelado de lenguajes. Los modelos complejos de redes neuronales - incluso más profundos que la LSTM que implementamos en otro notebook -, o incluso clásicos - como Naïve-Bayes - estaban lejos de ser realmente buenos en las predicciones. Como vimos, incluso tras un arduo preprocesamiento, los resultados no son tan óptimos como esperaríamos. En este contexto, en el mundo del procesamiento del lenguaje natural, han surgido varios modelos, como los tr ansformadores, el famoso BERT (*Bidirectional Encoder Representations from Transformers*, por sus siglas en inglés), ELMo, XL-Net y Open AI GPT-2, entre otros. Incluso con poco aprendizaje por transferencia y ajustes de hiperparámetros, pueden obtenerse excelentes resultados. [8]

Los avances en técnicas de transfer learning no han sido ajenos al campo de la medicina, donde han surgido modelos como SciBERT, BioBERT y ClinicalBERT. En este caso, haremos transfer learning a partir de uno de los modelos más recientes y de mayor éxito: BlueBERT. 

BlueBERT es un modelo de Machine Learning basado en BERT y desarrollado por Peng et. al [9], pre-entrenado sobre notas clínicas y abstracts de dos bases de datos muy conocidas en el mundo de la medicina: PubMed y MIMIC-III. 

A continuación, cargamos el modelo de BlueBERT y agregamos algunas capas al final (y descongelamos algunas de las últimas) con el fin de hacer transfer learning.

---
Lo primero que hacemos es cargar el modelo preentrenado:

In [23]:
pretrained_path = 'NCBI_BERT_pubmed_mimic_uncased_L-12_H-768_A-12'
config_path = os.path.join(pretrained_path, 'bert_config.json')
checkpoint_path = os.path.join(pretrained_path, 'bert_model.ckpt')
vocab_path = os.path.join(pretrained_path, 'vocab.txt')

Asimismo, definimos las constantes del modelo BERT:

In [4]:
SEQ_LEN = 128
BATCH_SIZE = 16
EPOCHS = 4
LR = 2e-5

### Preprocesamiento especial para BERT
Primero, cargamos el conjunto de textos.  Lo primero que tenemos que notar es que este algoritmo recibe una representación completamente diferente para su entrada. Por lo tanto, debemos hacer un preprocesamiento diferente. En este caso, seguiré la aproximación de [9]. En este caso, usaremos las funciones por defecto de Keras par BERT:

In [ ]:
datos_train = pd.read_csv('train_Data.csv')
X_train = datos_train['tokenized_abstracts']
Y_train = datos_train['problems_described']

In [68]:
tokenizer = BertTokenizer.from_pretrained('NCBI_BERT_pubmed_mimic_uncased_L-12_H-768_A-12')

In [75]:
input_ids = np.array([torch.tensor(tokenizer.encode(X_train[i])).unsqueeze(0) for i in range(len(X_train))])

/var/folders/gt/fh8j82dx4cj4lsdqnyn6l2fh0000gn/T/ipykernel_78019/3858949437.py:1: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  input_ids = np.array([torch.tensor(tokenizer.encode(X_train[i])).unsqueeze(0) for i in range(len(X_train))])
/var/folders/gt/fh8j82dx4cj4lsdqnyn6l2fh0000gn/T/ipykernel_78019/3858949437.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  inp

Ahora, cargamos el modelo pre-entrenado:

In [77]:
bert_model = load_trained_model_from_checkpoint(
  config_path,
  checkpoint_path,
  training=True,
  trainable=True,
  seq_len=SEQ_LEN,
)

Debido a que ya es un modelo de Keras, podemos revisar las capas que lo conforman. Vemos que son muchísimas, y a grandes rasgos podemos ver que se construye bajo una arquitectura de Encoder.

In [78]:
bert_model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Input-Token (InputLayer)       [(None, 128)]        0           []                               
                                                                                                  
 Input-Segment (InputLayer)     [(None, 128)]        0           []                               
                                                                                                  
 Embedding-Token (TokenEmbeddin  [(None, 128, 768),  23440896    ['Input-Token[0][0]']            
 g)                              (30522, 768)]                                                    
                                                                                                  
 Embedding-Segment (Embedding)  (None, 128, 768)     1536        ['Input-Segment[0][0]']    

                                                                                                  
 Encoder-3-MultiHeadSelfAttenti  (None, 128, 768)    0           ['Encoder-2-FeedForward-Norm[0][0
 on-Add (Add)                                                    ]',                              
                                                                  'Encoder-3-MultiHeadSelfAttentio
                                                                 n-Dropout[0][0]']                
                                                                                                  
 Encoder-3-MultiHeadSelfAttenti  (None, 128, 768)    1536        ['Encoder-3-MultiHeadSelfAttentio
 on-Norm (LayerNormalization)                                    n-Add[0][0]']                    
                                                                                                  
 Encoder-3-FeedForward (FeedFor  (None, 128, 768)    4722432     ['Encoder-3-MultiHeadSelfAttentio
 ward)    

 on-Dropout (Dropout)                                            n[0][0]']                        
                                                                                                  
 Encoder-6-MultiHeadSelfAttenti  (None, 128, 768)    0           ['Encoder-5-FeedForward-Norm[0][0
 on-Add (Add)                                                    ]',                              
                                                                  'Encoder-6-MultiHeadSelfAttentio
                                                                 n-Dropout[0][0]']                
                                                                                                  
 Encoder-6-MultiHeadSelfAttenti  (None, 128, 768)    1536        ['Encoder-6-MultiHeadSelfAttentio
 on-Norm (LayerNormalization)                                    n-Add[0][0]']                    
                                                                                                  
 Encoder-6

 Encoder-9-MultiHeadSelfAttenti  (None, 128, 768)    0           ['Encoder-9-MultiHeadSelfAttentio
 on-Dropout (Dropout)                                            n[0][0]']                        
                                                                                                  
 Encoder-9-MultiHeadSelfAttenti  (None, 128, 768)    0           ['Encoder-8-FeedForward-Norm[0][0
 on-Add (Add)                                                    ]',                              
                                                                  'Encoder-9-MultiHeadSelfAttentio
                                                                 n-Dropout[0][0]']                
                                                                                                  
 Encoder-9-MultiHeadSelfAttenti  (None, 128, 768)    1536        ['Encoder-9-MultiHeadSelfAttentio
 on-Norm (LayerNormalization)                                    n-Add[0][0]']                    
          

                                                                                                  
 Encoder-12-MultiHeadSelfAttent  (None, 128, 768)    0           ['Encoder-12-MultiHeadSelfAttenti
 ion-Dropout (Dropout)                                           on[0][0]']                       
                                                                                                  
 Encoder-12-MultiHeadSelfAttent  (None, 128, 768)    0           ['Encoder-11-FeedForward-Norm[0][
 ion-Add (Add)                                                   0]',                             
                                                                  'Encoder-12-MultiHeadSelfAttenti
                                                                 on-Dropout[0][0]']               
                                                                                                  
 Encoder-12-MultiHeadSelfAttent  (None, 128, 768)    1536        ['Encoder-12-MultiHeadSelfAttenti
 ion-Norm 

Es posible ver que el modelo tiene más de 109 millones de parámetros. En este caso, para poder realizar el transfer learning, hacemos ingeniería sobre las capas. Primero, extraemos las capas de entrada:

In [79]:
inputs = bert_model.inputs[:2]

Asimismo, tomamos la salida de la capa de salida (NSP-Dense):

In [80]:
dense = bert_model.get_layer('NSP-Dense').output

Finalmente, aquí es donde podemos empezar a considerar el tuneo de hiperparámetros. Con el fin de no cambiar los pesos ya entrenados de BERT, lo que hacemos es, al final, decidir si pasamos el resultado que obtenemos de BERT por algunas capas profundas para obtener más características. Asimismo, también podría ajustarse el número de neuronas de cada uno de estas capas adicionales que pretendan añadirse. No obstante, esto se sale del enfoque del curso, por lo que solo implementaremos un modelo básico.

Para el primer modelo básico, simplemente tomaremos el resultado de BERT y lo pasaremos por una capa densa con activación soft-max, característica de los problemas de clasificación, con 5 neuronas pues se tienen 5 clases.

In [81]:
outputs = keras.layers.Dense(units=5, activation='softmax')(dense)

Por último, compilamos nuestro modelo con las capas y vemos el resumen:

In [82]:
model = keras.models.Model(inputs, outputs)
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Input-Token (InputLayer)       [(None, 128)]        0           []                               
                                                                                                  
 Input-Segment (InputLayer)     [(None, 128)]        0           []                               
                                                                                                  
 Embedding-Token (TokenEmbeddin  [(None, 128, 768),  23440896    ['Input-Token[0][0]']            
 g)                              (30522, 768)]                                                    
                                                                                                  
 Embedding-Segment (Embedding)  (None, 128, 768)     1536        ['Input-Segment[0][0]']    

                                                                                                  
 Encoder-3-MultiHeadSelfAttenti  (None, 128, 768)    0           ['Encoder-2-FeedForward-Norm[0][0
 on-Add (Add)                                                    ]',                              
                                                                  'Encoder-3-MultiHeadSelfAttentio
                                                                 n-Dropout[0][0]']                
                                                                                                  
 Encoder-3-MultiHeadSelfAttenti  (None, 128, 768)    1536        ['Encoder-3-MultiHeadSelfAttentio
 on-Norm (LayerNormalization)                                    n-Add[0][0]']                    
                                                                                                  
 Encoder-3-FeedForward (FeedFor  (None, 128, 768)    4722432     ['Encoder-3-MultiHeadSelfAttentio
 ward)    

 on-Dropout (Dropout)                                            n[0][0]']                        
                                                                                                  
 Encoder-6-MultiHeadSelfAttenti  (None, 128, 768)    0           ['Encoder-5-FeedForward-Norm[0][0
 on-Add (Add)                                                    ]',                              
                                                                  'Encoder-6-MultiHeadSelfAttentio
                                                                 n-Dropout[0][0]']                
                                                                                                  
 Encoder-6-MultiHeadSelfAttenti  (None, 128, 768)    1536        ['Encoder-6-MultiHeadSelfAttentio
 on-Norm (LayerNormalization)                                    n-Add[0][0]']                    
                                                                                                  
 Encoder-6

 Encoder-9-MultiHeadSelfAttenti  (None, 128, 768)    0           ['Encoder-9-MultiHeadSelfAttentio
 on-Dropout (Dropout)                                            n[0][0]']                        
                                                                                                  
 Encoder-9-MultiHeadSelfAttenti  (None, 128, 768)    0           ['Encoder-8-FeedForward-Norm[0][0
 on-Add (Add)                                                    ]',                              
                                                                  'Encoder-9-MultiHeadSelfAttentio
                                                                 n-Dropout[0][0]']                
                                                                                                  
 Encoder-9-MultiHeadSelfAttenti  (None, 128, 768)    1536        ['Encoder-9-MultiHeadSelfAttentio
 on-Norm (LayerNormalization)                                    n-Add[0][0]']                    
          

                                                                                                  
 Encoder-12-MultiHeadSelfAttent  (None, 128, 768)    0           ['Encoder-12-MultiHeadSelfAttenti
 ion-Dropout (Dropout)                                           on[0][0]']                       
                                                                                                  
 Encoder-12-MultiHeadSelfAttent  (None, 128, 768)    0           ['Encoder-11-FeedForward-Norm[0][
 ion-Add (Add)                                                   0]',                             
                                                                  'Encoder-12-MultiHeadSelfAttenti
                                                                 on-Dropout[0][0]']               
                                                                                                  
 Encoder-12-MultiHeadSelfAttent  (None, 128, 768)    1536        ['Encoder-12-MultiHeadSelfAttenti
 ion-Norm 

Ahora, por defecto, BERT utiliza de optimizador el algoritmo de adam rectificado (RAdam), el cual es una variante del Adam estocástico que permite rectificar la varianza de la tasa de aprendizaje adaptativa. Asimismo, por la naturaleza de la vectorización y embedding que usa BERT, y al tratarse de un problema multiclase, usamos como función de pérdida la entropía categórica cruzada dispersa (sparse categorical cross entropy). Como métrica, volvemos a usar la precisión para ser consistentes con los modelos generados previamente:

In [83]:
model.compile(
      Adam(learning_rate=LR),
      loss='sparse_categorical_crossentropy',
      metrics=[keras.metrics.Precision(name='precision')],
  )


Finalmente, ha llegado la hora de correr el modelo. 

Asimismo, y de igual forma que el notebook anterior, volvemos a la variable objetivo una variable One-Hot:

In [84]:
Y_train = datos_train['problems_described']
Y_train = keras.utils.np_utils.to_categorical(Y_train)[:,1:]

In [85]:
history = model.fit(
    input_ids,
    Y_train, 
    epochs=EPOCHS, # Usamos las epocas por defecto que propone BERT
    batch_size=BATCH_SIZE, # El tamanio de los datos que se cargan en cada iteracion, propuesta por BERT
    validation_split=0.20, # Usamos una particion del conjunto de entrenamiento para medir la validacion
    shuffle=True,
)
#model.save('20_newsgroups.h5')

ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 2 array(s), for inputs ['Input-Token', 'Input-Segment'] but instead got the following list of 1 arrays: [array([[tensor([[ 101, 1031, 1005,  ..., 1005, 1033,  102]])],
       [tensor([[ 101, 1031, 1005,  ..., 1005, 1033,  102]])],
       [tensor([[ 101, 1031, 1005,  ..., 1005, 1033,  102]])],
       ......

## Bibliografía
---
[1] https://www.analyticsvidhya.com/blog/2021/06/lstm-for-text-classification/

[2] https://stanford.edu/~shervine/teaching/cs-230/cheatsheet-recurrent-neural-networks

[3] Application of Long Short-Term Memory (LSTM) Neural Network for Flood Forecasting - Scientific Figure on ResearchGate. Available from: https://www.researchgate.net/figure/The-structure-of-the-Long-Short-Term-Memory-LSTM-neural-network-Reproduced-from-Yan_fig8_334268507 [accessed 28 Mar, 2022]

[4] Zhang Y, Chen Q, Yang Z, Lin H, Lu Z. BioWordVec, improving biomedical word embeddings with subword information and MeSH. Scientific Data. 2019.

[5] https://towardsdatascience.com/simplified-math-behind-dropout-in-deep-learning-6d50f3f47275

[6] https://machinelearningmastery.com/how-to-choose-loss-functions-when-training-deep-learning-neural-networks/

[7] https://towardsdatascience.com/7-tips-to-choose-the-best-optimizer-47bb9c1219e

[8] https://medium.com/@manasmohanty/ncbi-bluebert-ncbi-bert-using-tensorflow-weights-with-huggingface-transformers-15a7ec27fc3d

[9]